In [1]:
import hashlib
import random

def generate_dsa_parameters():
    """
    Generate DSA parameters (p, q, g).
    For simplicity, these are small numbers, but in practice, p, q, and g should be much larger.
    """
    q = 19  # Small prime number
    p = 101  # Large prime such that (p-1) is divisible by q
    g = 2  # g < p, a primitive root modulo p
    return p, q, g

def generate_dsa_keys(p, q, g):
    """
    Generate public and private keys for DSA.
    """
    # Private key x (random number less than q)
    x = random.randint(1, q - 1)
    # Public key y = g^x mod p
    y = pow(g, x, p)
    return x, y

def sign_message(message, p, q, g, x):
    """
    Generate a digital signature for the given message using the private key x.
    """
    # Hash the message
    hash_value = int(hashlib.sha1(message.encode()).hexdigest(), 16) % q

    # Select a random integer k (1 < k < q)
    k = random.randint(1, q - 1)

    # Calculate r = (g^k mod p) mod q
    r = pow(g, k, p) % q
    if r == 0:
        return sign_message(message, p, q, g, x)  # Retry with a new k

    # Calculate s = k^(-1) * (hash + x * r) mod q
    k_inverse = pow(k, -1, q)
    s = (k_inverse * (hash_value + x * r)) % q
    if s == 0:
        return sign_message(message, p, q, g, x)  # Retry with a new k

    return r, s

def verify_signature(message, r, s, p, q, g, y):
    """
    Verify a digital signature (r, s) for the given message using the public key y.
    """
    # Check that r and s are valid
    if not (0 < r < q and 0 < s < q):
        return False

    # Hash the message
    hash_value = int(hashlib.sha1(message.encode()).hexdigest(), 16) % q

    # Calculate w = s^(-1) mod q
    w = pow(s, -1, q)

    # Calculate u1 = (hash * w) mod q and u2 = (r * w) mod q
    u1 = (hash_value * w) % q
    u2 = (r * w) % q

    # Calculate v = ((g^u1 * y^u2) mod p) mod q
    v = ((pow(g, u1, p) * pow(y, u2, p)) % p) % q

    # Signature is valid if v == r
    return v == r

# Example Usage
if __name__ == "__main__":
    # Generate DSA parameters
    p, q, g = generate_dsa_parameters()
    print(f"DSA Parameters: p = {p}, q = {q}, g = {g}")

    # Generate keys
    x, y = generate_dsa_keys(p, q, g)
    print(f"Private Key (x): {x}")
    print(f"Public Key (y): {y}")

    # Message to sign
    message = "Hello, DSA!"
    print(f"\nMessage: {message}")

    # Sign the message
    r, s = sign_message(message, p, q, g, x)
    print(f"Signature: r = {r}, s = {s}")

    # Verify the signature
    is_valid = verify_signature(message, r, s, p, q, g, y)
    print(f"\nIs the signature valid? {is_valid}")


DSA Parameters: p = 101, q = 19, g = 2
Private Key (x): 2
Public Key (y): 4

Message: Hello, DSA!
Signature: r = 16, s = 17

Is the signature valid? False
